# Question 2

In [20]:
import os  # Import to get api-key environment variable
import requests # Import to request data from ny times
import json
from time import sleep
from glob import glob
from collections import Counter
from string import punctuation
import itertools
import collections
import csv
from nltk.corpus import stopwords

In [4]:
# print (nytimes) # shows the nytimes api key that I used

33bbafed080c4b6c9cf9272dceb30985


In [3]:
nytimes = os.getenv('ny_times_api')

In [1]:
#booksearch = "https://api.nytimes.com/svc/books/v3/lists/"
#booksearch2 = "https://api.nytimes.com/svc/books/v3/lists/overview"
articlesearch = "https://api.nytimes.com/svc/search/v2/articlesearch.json" # article search api call

In [8]:
def getAllRequests(pageNumber): # article search
    request_params = { # request parameters dict used for the get method from the requests module
        "api-key"    : nytimes,
        "page"       : pageNumber
    }
    response = requests.get(articlesearch, request_params)
    sleep(10) # sleep so the nytimes api isn't pinged to fast
    with open('ArticleSearchPage' + str(pageNumber) + '.json', 'w', errors='ignore') as f:
        json.dump(response.json(), f, ensure_ascii=False)

#         "fq"         : 'source.contains:("New York Times")',

In [5]:
def getAllArchives(year):
    archivesearch = "https://api.nytimes.com/svc/archive/v1/" + str(year) + "/1.json" # archive search api call
    request_params = { # request parameters dict used for the get method from the requests module
        "api-key"    : nytimes
    }
    response = requests.get(archivesearch, request_params)
    sleep(10) # sleep so the nytimes api isn't pinged to fast
    with open('Archive' + str(year) + '.json', 'w', errors='ignore') as f:
        json.dump(response.json(), f, ensure_ascii=False)


In [20]:
i = 1
while i < 21:
    getAllRequests(i) # get article searches page 1-20
    i += 1

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>


In [7]:
i = 2010
while i < 2018:
    getAllArchives(i) # get archives search 2010-2017
    i += 1

<Response [200]>
<Response [200]>


In [49]:
all_articles = glob ('Article*') # get all article api json files

In [50]:
my_article_dict = dict() # dict for all json articles
for article in all_articles:
    file = open(article).read()
    json_file = json.loads(file)
    my_article_dict[file] = json_file

# First analysis of Articles:
- This analysis was to see which article had the most words on the first 20 pages of the articles api on 03/03/2017
- Each page had responses which contained documents
- Each document had a headline and a word count
- Using the two keys above, I was able to create a dictionary that was later sorted based on word count
- CONCLUSION:
    - "Inside Sara Berman's closet has the most words of any article written from the first 20 pages as of 03/03/2017

In [51]:
word_count_dict = dict()
for my_key in my_article_dict:
    for doc in my_article_dict[my_key]["response"]["docs"]:
        word_count_dict[doc['headline']['main']] = doc['word_count']
sorted_list = sorted(word_count_dict.items(), key=lambda pair: pair[1], reverse=True)
#print (sorted_list) 

In [52]:
ana1art = 'Ana1Articles'
if not os.path.exists(ana1art):
    os.mkdir(ana1art)
with open(ana1art + '/ArticlesWordCount.csv', 'w', newline='', ) as csvfile: # open the csvfile as a writeable file
    fieldname = ['word', 'frequency']
    writer = csv.DictWriter(csvfile, fieldnames=fieldname) # uses dictWriter to make columns in the csv file

    writer.writeheader()
    for i in sorted_list: # for to search through each dict pair in the list
        writer.writerow({'word': i[0], 'frequency': i[1]})

# Second Analysis of Articles API:
- There were 190 articles printed in the first 20 pages of the articles api on 03/03/2017
- Of that list: 70 were from the 'Associated Press', 62 were from 'Reuters', 56 were from 'The New York Times', 1 was from 'CNBC', and 1 was from 'Internation New York Times'
- This shows that the New York Times does not print as often as either the Associated Press (1st) or Reuters (2nd)
- CONCLUSION:
    - The New York Times writes only the third most articles, indicting either there is more staff at Associated Press and Reuters, or The New York Times does not bring in as much money as the first two elements

In [53]:
source_list = list()
for my_key in my_article_dict:
    for doc in my_article_dict[my_key]["response"]["docs"]:
        source_list.append(doc['source'])
source_set = set(source_list)

In [54]:
count_dict = dict()
for source in source_set:
    source_count = source_list.count(source)
    count_dict[source] = source_count
sorted_sources = sorted(count_dict.items(), key=lambda pair: pair[1], reverse=True)

In [56]:
ana2art = 'Ana2Articles'
if not os.path.exists(ana2art):
    os.mkdir(ana2art)
with open(ana2art + '/sorted_sources.csv', 'w', newline='', ) as csvfile: # open the csvfile as a writeable file
    fieldname = ['source', 'frequency']
    writer = csv.DictWriter(csvfile, fieldnames=fieldname) # uses dictWriter to make columns in the csv file

    writer.writeheader()
    for i in sorted_sources: # for to search through each dict pair in the list
        writer.writerow({'source': i[0], 'frequency': i[1]})

# Third Analysis of Articles API:
- This analysis was to see which section had the most articles written about it
- There was a total of 20 sections that had articles written about them
- The most commonly written about sections were: "U.S.", "World", and "Business Day"
- CONCLUSION:
    - "Most readers care more about the U.S. and World News than they do about all other news.

In [57]:
section_name_list = list()
for my_key in my_article_dict:
    for doc in my_article_dict[my_key]["response"]["docs"]:
        section_name_list.append(doc['section_name'])
section_name_set = set(section_name_list)
name_count = dict()
for name in section_name_set:
    name_count[name] = section_name_list.count(name)
sorted_section_names = sorted(name_count.items(), key=lambda pair: pair[1], reverse=True)

[('U.S.', 54), ('World', 41), ('Business Day', 26), ('Sports', 18), ('Arts', 10), ('Books', 6), ('Real Estate', 5), ('Technology', 4), ('Opinion', 4), ('T Magazine', 4), ('Fashion & Style', 3), ('Food', 3), ('Watching', 3), ('Movies', 2), ('Science', 2), ('Health', 1), ('Times Insider', 1), ('N.Y. / Region', 1), ('The Upshot', 1), ('Theater', 1)]


In [58]:
ana3art = 'Ana3Articles'
if not os.path.exists(ana3art):
    os.mkdir(ana3art)
with open(ana3art + '/sorted_section_names.csv', 'w', newline='', ) as csvfile: # open the csvfile as a writeable file
    fieldname = ['country', 'frequency']
    writer = csv.DictWriter(csvfile, fieldnames=fieldname) # uses dictWriter to make columns in the csv file

    writer.writeheader()
    for i in sorted_section_names: # for to search through each dict pair in the list
        writer.writerow({'country': i[0], 'frequency': i[1]})

# First Archive Analysis
- Second analysis of the archive files was to show zipfs law with stopwords
- Conclusion:
    - The graph that is saved as: zipfs_archives_2010-2017_withstopwords.png confirms that zipfs law still holds true.  After doing a log-log plot it shows that the most common word is used about twice as frequently as the next word and so on, until the end where all words are used once.

In [13]:
all_archives = glob ('Archive*') # get all article api json files
#print (all_archives)

In [14]:
my_archive_dict = dict() # dict for all json articles
for archive in all_archives: # read in each file from the Archive list
    file = open(archive).read()
    json_file = json.loads(file)
    my_archive_dict[file] = json_file

In [15]:
my_snippet_dict = dict()
for file in my_archive_dict: # pull out each file in the dictionary
     for doc in my_archive_dict[file]["response"]["docs"]:
            if not(doc['snippet'] == None):
                my_snippet_dict[doc['headline']['main']] = (doc['snippet'].lower())

In [16]:
def removePunctuation (a_str):
    for a_char in a_str:
        if a_char in punctuation:
            a_str = a_str.replace(a_char, '')
    return a_str

In [17]:
my_word_list = list()
my_word_counter = dict()
for snippet in my_snippet_dict: # go through each snippet in the dictionary
    my_snippet_dict[snippet] = removePunctuation(my_snippet_dict[snippet]) # remove punctuation from each string
    my_word_list.append(my_snippet_dict[snippet].split()) # add each word to the end of the list
    
word_frequency = collections.defaultdict(int)
for x in itertools.chain.from_iterable(my_word_list):
    word_frequency[x] += 1
    
sorted_word_count = sorted(word_frequency.items(), key=lambda pair: pair[1], reverse=True) # sort the list

In [19]:
ana1arch = 'Ana1Archive'
if not os.path.exists(ana1arch):
    os.mkdir(ana1arch)
counter = 1
with open(ana1arch + '/zipfs_archives_2010-2017.csv', 'w', newline='', ) as csvfile: # open the csvfile as a writeable file
    fieldname = ['word', 'rank', 'frequency']
    writer = csv.DictWriter(csvfile, fieldnames=fieldname) # uses dictWriter to make columns in the csv file

    writer.writeheader()
    for i in sorted_word_count: # for to search through each dict pair in the list
        writer.writerow({'word': i[0], 'rank': counter, 'frequency': i[1]})
        counter += 1

# Second Archive Analysis
- Second analysis of the archive files was to show zipfs law without stopwords
- Conclusion:
    - The graph that is saved as: zipfs_archives_no_stopwords_2010-2017.png confirms that zipfs law still holds true.  After doing a log-log plot it shows that the most common word is used about twice as frequently as the next word and so on, until the end where all words are used once.

In [21]:
all_archives = glob ('Archive*') # get all article api json files
#print (all_archives)

In [22]:
my_archive_dict = dict() # dict for all json articles
for archive in all_archives: # read in each file from the Archive list
    file = open(archive).read()
    json_file = json.loads(file)
    my_archive_dict[file] = json_file

In [45]:
my_word_list = list()
my_word_counter = dict()
for snippet in my_snippet_dict: # go through each snippet in the dictionary
    my_snippet_dict[snippet] = removePunctuation(my_snippet_dict[snippet]) # remove punctuation from each string
    my_word_list.append(my_snippet_dict[snippet].split()) # add each word to the end of the list

my_word_list_no_stopwords = list()
my_word_list_no_stopwords = removeStopWords(my_word_list)
    
word_frequency = collections.defaultdict(int)
for x in itertools.chain.from_iterable(my_word_list_no_stopwords):
    word_frequency[x] += 1
    
sorted_word_count = sorted(word_frequency.items(), key=lambda pair: pair[1], reverse=True) # sort the list

In [23]:
def removePunctuation (a_str):
    for a_char in a_str:
        if a_char in punctuation:
            a_str = a_str.replace(a_char, '')
    return a_str

In [44]:
def removeStopWords (alist):
    for my_list in alist:
        if my_list:
            for word in my_list: 
                if word in stopwords.words('english'):
                    my_list.remove(word)
    return alist

In [46]:
ana2arch = 'Ana2Archive'
if not os.path.exists(ana2arch):
    os.mkdir(ana2arch)
counter = 1
with open(ana2arch + '/zipfs_archives_no_stopwords_2010-2017.csv', 'w', newline='', ) as csvfile: # open the csvfile as a writeable file
    fieldname = ['word', 'rank', 'frequency']
    writer = csv.DictWriter(csvfile, fieldnames=fieldname) # uses dictWriter to make columns in the csv file

    writer.writeheader()
    for i in sorted_word_count: # for to search through each dict pair in the list
        writer.writerow({'word': i[0], 'rank': counter, 'frequency': i[1]})
        counter += 1

# Third Archive Analysis
- Third analysis of the archive files was to count the number contributions each author made to see who deserves a raise
- This analysis required looking at all archives from 2010-present and looking at all of the contributors in the byline
- After getting the list of contributors, counting how many instances of their names was next.
- CONCLUSION:
    - Paul Krugman had his name against the most stories (665), so he is going to get a raise

In [4]:
all_archives = glob ('Archive*') # get all article api json files

my_archive_dict = dict() # dict for all json articles
for archive in all_archives: # read in each file from the Archive list
    file = open(archive).read()
    json_file = json.loads(file)
    my_archive_dict[file] = json_file

In [5]:
my_name_list = list()
for file in my_archive_dict: # pull out each file in the dictionary
     for doc in my_archive_dict[file]["response"]["docs"]:
        if (doc['byline']) and (doc['byline']['person']):
            for people in doc['byline']['person']:
                if ('lastname' in people.keys()):
                    my_name_list.append(people['firstname'].lower() + ' ' + people['lastname'].lower())

In [6]:
#print (my_name_list)
name_set = set(my_name_list)

In [7]:
name_count = dict()
for name in name_set:
    name_count[name] = my_name_list.count(name)

In [11]:
sorted_name_count = sorted(name_count.items(), key=lambda pair: pair[1], reverse=True)
ana3arch = 'Ana3Archive'
if not os.path.exists(ana3arch):
    os.mkdir(ana3arch)
counter = 1
with open(ana3arch + '/sorted_name.csv', 'w', newline='', ) as csvfile: # open the csvfile as a writeable file
    fieldname = ['author', 'rank', 'contributions']
    writer = csv.DictWriter(csvfile, fieldnames=fieldname) # uses dictWriter to make columns in the csv file

    writer.writeheader()
    for i in sorted_name_count: # for to search through each dict pair in the list
        writer.writerow({'author': i[0], 'rank': counter, 'contributions': i[1]})
        counter += 1